In [1]:
import pandas as pd
df=pd.read_csv(r"C:\Users\Arun\Downloads\netflix_titles.csv\netflix_titles.csv")
df.head()
df=df[df["rating"].notna()]
print(df.isnull().sum())
df["release_year"] = df["release_year"].astype(str)  
print(df.dtypes)


show_id            0
type               0
title              0
director        2631
cast             825
country          830
date_added        10
release_year       0
rating             0
duration           3
listed_in          0
description        0
dtype: int64
show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year    object
rating          object
duration        object
listed_in       object
description     object
dtype: object


In [2]:
rej=["country","date_added","duration"]
for i in rej:
    df=df.drop(i,axis=1)
df["new"]=df["director"]+df["cast"]    
df=df.fillna(" ")
df['new']=df["new"].apply(lambda x : x.replace(" ","") )
df["new"].head(5)


0                                                     
1                                                     
2    JulienLeclercqSamiBouajila,TracyGotoas,SamuelJ...
3                                                     
4                                                     
Name: new, dtype: object

In [3]:
df.head()

,show_id,type,title,director,cast,release_year,rating,listed_in,description,new
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,,2020,PG-13,Documentaries,"As her father nears the end of his life, filmm...",
1,s2,TV Show,Blood & Water,,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",2021,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,"JulienLeclercqSamiBouajila,TracyGotoas,SamuelJ..."
3,s4,TV Show,Jailbirds New Orleans,,,2021,TV-MA,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",
4,s5,TV Show,Kota Factory,,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",2021,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,


In [4]:
df["tags"] = df[["type", "new", "release_year", "rating", "listed_in", "description"]].agg(lambda x: ' '.join(x.astype(str)), axis=1)


In [5]:
df = df[['show_id', 'title', 'tags']]
df['tags']=df['tags'].apply(lambda x :x.lower())
df.head()
import nltk
from nltk.stem import WordNetLemmatizer
def stem(text):
    y=[]
    nltk.download('wordnet')
    for i in text.split(" "):
        model=WordNetLemmatizer()
        y.append(model.lemmatize(i))
        return " ".join(y)

# df['tags']=df['tags'].apply(stem)




In [6]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
cv=TfidfVectorizer(max_features=5000,stop_words="english",)
vector = cv.fit_transform(df["tags"]).toarray()
vector


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [7]:
cv.get_feature_names_out()[:500]

array(['000', '10', '100', '11', '12', '13', '14', '15', '16', '17', '18',
       '18th', '19', '1920s', '1930s', '1944', '1945', '1950s', '1960',
       '1960s', '1962', '1967', '1968', '1969', '1970s', '1971', '1972',
       '1973', '1974', '1975', '1976', '1977', '1978', '1979', '1980',
       '1980s', '1981', '1982', '1983', '1984', '1985', '1986', '1987',
       '1988', '1989', '1990', '1990s', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '19th', '20', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016',
       '2017', '2018', '2019', '2020', '2021', '20s', '20th', '21st',
       '24', '25', '28', '30', '40', '50', '500', '50th', '60', '70s',
       '80s', '90s', 'abandoned', 'abandons', 'abducted',
       'abigailbreslin', 'abilities', 'ability', 'able', 'aboard',
       'abroad', 'absurd', 'abuse', 'abusive', 'academic', 'academy',
       'accept', '

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
similarity=cosine_similarity(vector)

In [9]:
def recommend(movie):
    movie=df[df['title']==movie].index[0]
    distance=similarity[movie]
    movie_list=sorted(list(enumerate(distance)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movie_list:
        print(df.iloc[i[0]].title)
       
    

In [10]:
recommend("Kota Factory")

O-Negative, Love Can’t Be Designed
Single Ladies Senior
Feels Like Ishq
Titipo Titipo
Bad Guys
